In [22]:
from datetime import datetime, timedelta

# News in now-timespan will be kept in database 
now = datetime.now()
timespan = timedelta(days=30)

In [19]:
# newsapi.ai key: 0215dfd8-3e70-4f43-868f-6991882ac84f
# 100 items = 1 token, 2000 total tokens per month
from eventregistry import *

er = EventRegistry(apiKey = '0215dfd8-3e70-4f43-868f-6991882ac84f')
query = {
    "$query": {
        "$and": [
            {
                "categoryUri": "news/Politics"
            },
            {
                "dateStart": "2023-11-19",
                "dateEnd": "2023-11-26",
#                 "dateStart": f"{(now.date()-timedelta(days=1)).strftime("%Y-%m-%d")}",
#                 "dateEnd": f"{now.date().strftime("%Y-%m-%d")}",
                "lang": "eng"
            }
        ]
    },
    "$filter": {
        "startSourceRankPercentile": 0,
        "endSourceRankPercentile": 10
    }
}
q = QueryArticlesIter.initWithComplexQuery(query)
# change maxItems to get the number of results that you want
articles = []
for article in q.execQuery(er, maxItems=10000):
    articles.append(article)

In [23]:
import json

with open('news_database.json', 'r') as json_file:
    news_database = json.load(json_file)
for a in articles:
    if a['title'] not in news_database.keys():
        news_database[a['title']] = [a['date'], a['title'], a['source'], a['url'], a['body']]
for key in news_database: # Remove old news
    if datetime.strptime(news_database[key][0], '%Y-%m-%d') < now - timespan: 
        del news_database[key]
out_json = json.dumps(news_database, indent=4)
with open('news_database.json', 'w') as outfile:
    outfile.write(out_json)